In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

In [14]:
import pandas as pd

dataset = pd.read_csv(
    "/content/drive/MyDrive/NLP-Course/Comment-Classification/comments.tsv", sep="\t"
)
print(dataset.shape)

random_samples = dataset.sample(10)
print(random_samples)

(1000, 2)
     should_ban                                       comment_text
912           0  "\n\n Please do not vandalize pages, as you di...
703           0  Some new findings to be reflected in the main ...
444           1  WHY THE FUCK ARE THESE TWO THE SAME? THEY ARE ...
330           0  What is wrong with a neutrally coloured map sh...
200           1  He was a racial Jew, you fool. If you want to ...
253           0  Thank you so much sir. It's a very strong feel...
52            0  "\n\nWhilst I don't agree with your reasoning,...
645           0  Tidying up \n\nI applied for Highbeam access h...
468           0  "\nComment It's official: this stays as ""Hole...
819           1  is dope you.  Hey Rick James, leave Rick James...


In [15]:
normal = dataset[dataset["should_ban"] == 0]
toxic = dataset[dataset["should_ban"] == 1]
print("normal Percentage =", (len(normal) / len(dataset)) * 100, "%")
print("toxic Percentage =", (len(toxic) / len(dataset)) * 100, "%")

normal Percentage = 49.2 %
toxic Percentage = 50.8 %


In [16]:
# Splitting the data into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(
    dataset["comment_text"],
    dataset["should_ban"],
    test_size=0.3,
    random_state=42,
    shuffle=True,
)

# Checking the sizes of train and test sets
print("Train data size:", train_data.shape[0])
print("Test data size:", test_data.shape[0])

Train data size: 700
Test data size: 300


In [20]:
# Step 1: Data Preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r"[^\w\s]", "", text)
    return text


# Preprocess train emails
train_data = [preprocess_text(comment_text) for comment_text in train_data]

# Preprocess test emails
test_data = [preprocess_text(comment_text) for comment_text in test_data]

train_data[0]

'will do maralia i edited as i saw it but i supposed was following my own hunch'

In [ ]:
# Tokenize and convert text data into TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
train_data_tfidf = tfidf_vectorizer.fit_transform(train_data)
test_data_tfidf = tfidf_vectorizer.transform(test_data)

In [33]:
# Create and train a Multinomial Naive Bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(train_data_tfidf, train_labels)

MultinomialNB()

In [34]:
# Predict on the test set
y_pred = naive_bayes_classifier.predict(test_data_tfidf)

# Evaluate the classifier
accuracy = accuracy_score(test_labels, y_pred)
report = classification_report(test_labels, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.8033333333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.81      0.78       129
           1       0.85      0.80      0.82       171

    accuracy                           0.80       300
   macro avg       0.80      0.80      0.80       300
weighted avg       0.81      0.80      0.80       300

